<a href="https://colab.research.google.com/github/DevKor-Team/MRDC-Challenge/blob/jo/jo/week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
from __future__ import print_function
from __future__ import division

from PIL import Image

import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
# from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import random_split

import matplotlib.pyplot as plt
import itertools    # confusion matrix에서 사용

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import albumentations
import albumentations.pytorch

In [3]:
BASE_DIR = '/content/gdrive/MyDrive/ML/MRDC-competition/data'

import os

train = pd.read_csv(os.path.join(BASE_DIR, 'Train.csv'))
print(train.shape)
train.head()

(5340, 2)


,Image_id,Label
0,id_004wknd7qd.jpg,blast
1,id_004wknd7qd_rgn.jpg,blast
2,id_005sitfgr2.jpg,brown
3,id_005sitfgr2_rgn.jpg,brown
4,id_00stp9t6m6.jpg,blast


In [4]:
train_rgb = train.loc[~train['Image_id'].str.contains('_rgn')]
train_rgb.head()

,Image_id,Label
0,id_004wknd7qd.jpg,blast
2,id_005sitfgr2.jpg,brown
4,id_00stp9t6m6.jpg,blast
6,id_012zxewnhx.jpg,blast
8,id_0186qwq2at.jpg,healthy


In [5]:
test = pd.read_csv(os.path.join(BASE_DIR, 'Test.csv'))
test_rgb = test.loc[~test['Image_id'].str.contains('_rgn')]
test_rgb.head()

,Image_id
0,id_00vl5wvxq3.jpg
2,id_01hu05mtch.jpg
4,id_030ln10ewn.jpg
6,id_03z57m8xht.jpg
8,id_04ngep1w4b.jpg


In [6]:
ss = pd.read_csv(os.path.join(BASE_DIR, 'SampleSubmission.csv'))
ss.head()

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.0,0.0,0.0
1,id_01hu05mtch.jpg,0.0,0.0,0.0
2,id_030ln10ewn.jpg,0.0,0.0,0.0
3,id_03z57m8xht.jpg,0.0,0.0,0.0
4,id_04ngep1w4b.jpg,0.0,0.0,0.0


In [7]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [8]:
def train_model(device, model, train_loader, val_loader, criterion, optimizer, num_epochs=5, early_stopping=None):
    model = model.to(device)

    dl = {'train': train_loader,
          'val': val_loader}
    
    val_label = []
    val_pred = []

    val_loss = 0.0
    for epoch in range(num_epochs):
        val_label = []
        val_pred = []

        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dl[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients -> backward시 필요
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)
                    # print(preds)
                    # print(labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    if phase == 'val':    # and epoch == num_epochs - 1:
                        val_label += labels.tolist()
                        val_pred += preds.tolist()
                        # print(val_label)

                # statistics
                running_loss += loss.item() * inputs.size(0)  # size(0) : batch size (첫 번째 차원 개수)
                                                              # item() : tensor에서 저장된 값만 가져오기
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dl[phase].dataset)  # 이렇게 나누면 epoch당 평균 loss가 됨
            epoch_acc = running_corrects.double() / len(dl[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            val_loss = epoch_loss

        if early_stopping != None:
          early_stopping(val_loss, model) # epoch_loss에는 validation loss가 저장
          if early_stopping.early_stop:
            print("Early stopping")
            break

    # load best model weights
    model.load_state_dict(torch.load(early_stopping.path))
    return model, val_label, val_pred

In [9]:
def test_model(device, model, test_loader):
    test_pred = []

    model.eval()
    model = model.to(device)
    with torch.set_grad_enabled(False):
        for features in test_loader:
            features = features.to(device)

            outputs = model(features.to(torch.float))
            probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
            test_pred.append(probabilities.tolist())

    return test_pred

In [ ]:
# confusion matrix 시각화
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, target_names=None, labels=True):
    accuracy = np.trace(cm) / float(np.sum(cm))

    cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(9, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    thresh = cm.max() / 2

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names)
        plt.yticks(tick_marks, target_names)

    if labels:
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, "{:,}".format(cm[i, j]), horizontalalignment="center",
                     color="white" if cm[i,j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [24]:
class Img_Dataset_with_aug(Dataset):
    def __init__(self, file_path, aug_transform, table, is_train=True, test_transform=None):
        self.file_path = file_path
        self.aug_transform = aug_transform
        self.table = table
        self.is_train = is_train
        self.test_transform = test_transform
        
        self.img_name_list = self.table['Image_id'].tolist()
        self.img_list = []

        if self.is_train:   # train -> augmentation 필요
          self.table_label_list = self.table['Label'].tolist()
          self.label_list = []
          for i, img_name in enumerate(self.img_name_list):
            img = Image.open(os.path.join(self.file_path, img_name))
            print(i)
            if self.table_label_list[i] == 'blast':
              img_transformed = self.test_transform(image=np.array(img))['image']
              self.img_list.append(img_transformed)
              self.label_list.append(0)
            elif self.table_label_list[i] == 'brown':
              for _ in range(2):   
                img_transformed = self.aug_transform(image=np.array(img))['image']
                self.img_list.append(img_transformed)
                self.label_list.append(1)
            else:
              for _ in range(4):   
                img_transformed = self.aug_transform(image=np.array(img))['image']
                self.img_list.append(img_transformed)
                self.label_list.append(2)

        else:   # test -> augmentation 필요X
            for img_name in self.img_name_list:
                img = Image.open(os.path.join(self.file_path, img_name))
                img_transformed = self.test_transform(image=np.array(img))['image']
                self.img_list.append(img_transformed)
  
    def __len__(self):
        return len(self.img_list)
  
    def __getitem__(self, index):
        if self.is_train:
            return self.img_list[index], self.label_list[index]
        else:
            return self.img_list[index]

In [19]:
def make_data_loader_with_aug(batch_size=128, size=224, split=0.8):
  IMG_DIR = '/content/gdrive/MyDrive/ML/MRDC-competition/data/images'
  aug_transform=albumentations.Compose([
  albumentations.Resize(size, size),
  albumentations.OneOf([
                        albumentations.HorizontalFlip(p=1),
                        albumentations.RandomRotate90(p=1),
                        albumentations.VerticalFlip(p=1)            
  ], p=0.5),
  albumentations.Normalize(),
  albumentations.pytorch.transforms.ToTensorV2(),
])
  test_transform=albumentations.Compose([
    albumentations.Resize(size, size),
    albumentations.Normalize(),
    albumentations.pytorch.transforms.ToTensorV2(),
  ])

  # train data
  train_dataset = Img_Dataset_with_aug(IMG_DIR, aug_transform, train_rgb, test_transform=test_transform)

  train_size = int(len(train_dataset) * split)
  val_size = len(train_dataset) - train_size

  train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=2)
  
  # test data
  test_dataset = Img_Dataset_with_aug(IMG_DIR, aug_transform, test_rgb, is_train=False, test_transform=test_transform)
  test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
  
  return train_loader, val_loader, test_loader

In [ ]:
# Cuda out of memory error 대비
# 이거 한 번 돌리고 batch size 줄여서 위의 셀 다시 한 번 돌려주기
import gc

gc.collect()
torch.cuda.empty_cache()

# ConvNext-Base 384

In [12]:
import sys
# check whether run in Colab
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !pip3 install timm==0.5.4 

from timm import create_model

Running in Colab.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 4.1 MB/s 


In [ ]:
import timm
avail_pretrained_models = timm.list_models(pretrained=True)
len(avail_pretrained_models), avail_pretrained_models

(592,
 ['adv_inception_v3',
  'bat_resnext26ts',
  'beit_base_patch16_224',
  'beit_base_patch16_224_in22k',
  'beit_base_patch16_384',
  'beit_large_patch16_224',
  'beit_large_patch16_224_in22k',
  'beit_large_patch16_384',
  'beit_large_patch16_512',
  'botnet26t_256',
  'cait_m36_384',
  'cait_m48_448',
  'cait_s24_224',
  'cait_s24_384',
  'cait_s36_384',
  'cait_xs24_384',
  'cait_xxs24_224',
  'cait_xxs24_384',
  'cait_xxs36_224',
  'cait_xxs36_384',
  'coat_lite_mini',
  'coat_lite_small',
  'coat_lite_tiny',
  'coat_mini',
  'coat_tiny',
  'convit_base',
  'convit_small',
  'convit_tiny',
  'convmixer_768_32',
  'convmixer_1024_20_ks9_p14',
  'convmixer_1536_20',
  'convnext_base',
  'convnext_base_384_in22ft1k',
  'convnext_base_in22ft1k',
  'convnext_base_in22k',
  'convnext_large',
  'convnext_large_384_in22ft1k',
  'convnext_large_in22ft1k',
  'convnext_large_in22k',
  'convnext_small',
  'convnext_tiny',
  'convnext_xlarge_384_in22ft1k',
  'convnext_xlarge_in22ft1k',
  'c

In [ ]:
model_name = 'convnext_base_384_in22ft1k'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device = ", device)
model = create_model(model_name, pretrained=True).to(device)

device =  cuda


Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_1k_384.pth" to /root/.cache/torch/hub/checkpoints/convnext_base_22k_1k_384.pth


In [ ]:
print(model)

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU(approximate=none)
            (drop1): Dropout(p=0.0, inplace=False)
            (fc2): Linear(in_features=512, out_features=128, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (norm): LayerNorm((128,), e

In [ ]:
class myConvNextBase384(nn.Module):
  def __init__(self, num_classes):
    super(myConvNextBase384, self).__init__()
    self.model_ft = create_model('convnext_base_384_in22ft1k', pretrained=True)
    
    num_ftrs = self.model_ft.head.fc.in_features
    self.model_ft.head.fc = nn.Linear(num_ftrs, num_classes)

  def forward(self, x):
      out = self.model_ft(x)
      return out

In [ ]:
convnext_base_384 = myConvNextBase384(3)

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_1k_384.pth" to /root/.cache/torch/hub/checkpoints/convnext_base_22k_1k_384.pth


In [ ]:
# train
train_loader, val_loader, test_loader = make_data_loader_with_aug(batch_size=8, size=384)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(convnext_base_384.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
MODEL_DIR = '/content/gdrive/MyDrive/ML/MRDC-competition/models'
early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.00001, path=os.path.join(MODEL_DIR, "convnext_base_384_aug_checkpoint.pt"))

In [ ]:
train_rgb['Label'].value_counts()

blast      1494
brown       766
healthy     410
Name: Label, dtype: int64

In [ ]:
convnext_base_384, val_label, val_pred = train_model(device, convnext_base_384, train_loader, val_loader, criterion, optimizer, 30, early_stopping)

Epoch 1/30
----------
train Loss: 0.5779 Acc: 0.7591
val Loss: 0.4516 Acc: 0.8287
Validation loss decreased (inf --> 0.451625).  Saving model ...
Epoch 2/30
----------
train Loss: 0.1935 Acc: 0.9354
val Loss: 0.1564 Acc: 0.9550
Validation loss decreased (0.451625 --> 0.156423).  Saving model ...
Epoch 3/30
----------
train Loss: 0.1154 Acc: 0.9587
val Loss: 0.0771 Acc: 0.9722
Validation loss decreased (0.156423 --> 0.077071).  Saving model ...
Epoch 4/30
----------
train Loss: 0.0731 Acc: 0.9762
val Loss: 0.1650 Acc: 0.9315
EarlyStopping counter: 1 out of 5
Epoch 5/30
----------
train Loss: 0.0428 Acc: 0.9853
val Loss: 0.0724 Acc: 0.9807
Validation loss decreased (0.077071 --> 0.072402).  Saving model ...
Epoch 6/30
----------
train Loss: 0.0886 Acc: 0.9689
val Loss: 0.1455 Acc: 0.9572
EarlyStopping counter: 1 out of 5
Epoch 7/30
----------
train Loss: 0.0445 Acc: 0.9842
val Loss: 0.4734 Acc: 0.8844
EarlyStopping counter: 2 out of 5
Epoch 8/30
----------
train Loss: 0.0480 Acc: 0.9847


In [ ]:
classes = ['blast', 'brown', 'healthy']
plot_confusion_matrix(confusion_matrix(val_label, val_pred), target_names=classes)

NameError: ignored

In [ ]:
# convnext_base_384.load_state_dict(torch.load(os.path.join(MODEL_DIR, "convnext_base_384_aug_checkpoint.pt")))
test_pred = test_model(device, convnext_base_384, test_loader)

In [ ]:
ss.loc[:, ['blast', 'brown', 'healthy']] = test_pred
ss.to_csv("result_convnext_base384_aug2.csv", index=False)
ss.head()

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.999346,0.000636,0.000018
1,id_01hu05mtch.jpg,0.124495,0.875465,0.000039
2,id_030ln10ewn.jpg,0.887034,0.112725,0.000242
3,id_03z57m8xht.jpg,0.999986,0.000011,0.000003
4,id_04ngep1w4b.jpg,0.995631,0.004227,0.000141


score : 0.19 (SOTA) 근데 너무 오래 걸림..ㅎ

# Target 별 Loss 구하는 코드

In [ ]:
def val_model(device, model, val_loader):
    test_pred = []

    model.eval()
    model = model.to(device)
    val_target = []
    with torch.set_grad_enabled(False):
        for features, target in val_loader:
            features = features.to(device)
            val_target += target.tolist()

            outputs = model(features.to(torch.float))
            probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
            test_pred.append(probabilities.tolist())

    return test_pred, val_target

In [ ]:
val_loader = DataLoader(val_loader.dataset, batch_size=1, shuffle=True, num_workers=2)

In [ ]:
val_pred, val_target = val_model(device, convnext_base_384, val_loader)

In [ ]:
from sklearn.metrics import log_loss

def log_loss_by_label(val_target, val_pred):
  val_0 = []
  val_1 = []
  val_2 = []

  for i, target in enumerate(val_target):
    if target == 0:
      val_0.append(val_pred[i])
    elif target == 1:
      val_1.append(val_pred[i])
    elif target == 2:
      val_2.append(val_pred[i])
  print("blast :")
  print(log_loss([[1, 0, 0] for _ in range(len(val_0))], val_0))
  print("brown :")
  print(log_loss([[0, 1, 0] for _ in range(len(val_1))], val_1))
  print("healthy :")
  print(log_loss([[0, 0, 1] for _ in range(len(val_2))], val_2))

In [ ]:
log_loss_by_label(val_target, val_pred)

blast :
0.1029236394833858
brown :
0.18469007093496678
healthy :
0.036418428098195094


## ResNext에서 Target 별 log loss 구하기

In [ ]:
class myResNeXt50(nn.Module):
  def __init__(self, num_classes):
    super(myResNeXt50, self).__init__()
    self.model_ft = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.DEFAULT)
    
    num_ftrs = self.model_ft.fc.in_features
    self.model_ft.fc = nn.Linear(num_ftrs, num_classes)

  def forward(self, x):
      out = self.model_ft(x)
      return out

In [ ]:
resnext50_tf = myResNeXt50(3)

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-1a0047aa.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-1a0047aa.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

In [ ]:
resnext50_tf.load_state_dict(torch.load(os.path.join(MODEL_DIR, "resnext50-aug_checkpoint.pt")))

<All keys matched successfully>

In [ ]:
val_pred, val_target = val_model(device, resnext50_tf, val_loader)

In [ ]:
log_loss_by_label(val_target, val_pred)

blast :
0.11601280693742066
brown :
0.3763652798551647
healthy :
0.4659342289541602


# SEResNext50_32x4d

In [ ]:
m = create_model('seresnext50_32x4d', pretrained=True)
print(m)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/seresnext50_32x4d_racm-a304a460.pth" to /root/.cache/torch/hub/checkpoints/seresnext50_32x4d_racm-a304a460.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (se): SEModule(
        (fc1): Conv2

In [ ]:
class mySEResNext50(nn.Module):
  def __init__(self, num_classes):
    super(mySEResNext50, self).__init__()
    self.model_ft = create_model('seresnext50_32x4d', pretrained=True)
    
    num_ftrs = self.model_ft.fc.in_features
    self.model_ft.fc = nn.Linear(num_ftrs, num_classes)

  def forward(self, x):
      out = self.model_ft(x)
      return out

In [ ]:
seresnext50_tf = mySEResNext50(3)

In [ ]:
# train
train_loader, val_loader, test_loader = make_data_loader_with_aug(batch_size=32, size=224)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(seresnext50_tf.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
MODEL_DIR = '/content/gdrive/MyDrive/ML/MRDC-competition/models'
early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.00001, path=os.path.join(MODEL_DIR, "seresnext50_aug_checkpoint.pt"))

In [ ]:
seresnext50_tf, val_label, val_pred = train_model(device, seresnext50_tf, train_loader, val_loader, criterion, optimizer, 30, early_stopping)

Epoch 1/30
----------
train Loss: 0.4925 Acc: 0.8087
val Loss: 0.1685 Acc: 0.9379
Validation loss decreased (inf --> 0.168491).  Saving model ...
Epoch 2/30
----------
train Loss: 0.1178 Acc: 0.9614
val Loss: 0.1589 Acc: 0.9443
Validation loss decreased (0.168491 --> 0.158917).  Saving model ...
Epoch 3/30
----------
train Loss: 0.0628 Acc: 0.9842
val Loss: 0.1029 Acc: 0.9625
Validation loss decreased (0.158917 --> 0.102941).  Saving model ...
Epoch 4/30
----------
train Loss: 0.0316 Acc: 0.9909
val Loss: 0.0998 Acc: 0.9647
Validation loss decreased (0.102941 --> 0.099754).  Saving model ...
Epoch 5/30
----------
train Loss: 0.0399 Acc: 0.9858
val Loss: 0.1079 Acc: 0.9668
EarlyStopping counter: 1 out of 5
Epoch 6/30
----------
train Loss: 0.0487 Acc: 0.9837
val Loss: 0.1585 Acc: 0.9518
EarlyStopping counter: 2 out of 5
Epoch 7/30
----------
train Loss: 0.0420 Acc: 0.9855
val Loss: 0.1078 Acc: 0.9668
EarlyStopping counter: 3 out of 5
Epoch 8/30
----------
train Loss: 0.0203 Acc: 0.9944


In [ ]:
classes = ['blast', 'brown', 'healthy']
plot_confusion_matrix(confusion_matrix(val_label, val_pred), target_names=classes)

In [ ]:
test_pred = test_model(device, seresnext50_tf, test_loader)

In [ ]:
ss.loc[:, ['blast', 'brown', 'healthy']] = test_pred
ss.to_csv("result_seresnext_aug.csv", index=False)
ss.head()

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.999999,1.147116e-07,4.949978e-07
1,id_01hu05mtch.jpg,0.000175,9.996787e-01,1.459547e-04
2,id_030ln10ewn.jpg,0.990332,1.391934e-03,8.276543e-03
3,id_03z57m8xht.jpg,1.000000,4.183470e-11,5.567445e-11
4,id_04ngep1w4b.jpg,0.998910,1.761415e-05,1.072641e-03


score : 0.25

# Swin tiny patch4 window7 224

In [ ]:
m = create_model('swin_tiny_patch4_window7_224', pretrained=True)
print(m)

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth" to /root/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth


SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): Sequential(
    (0): BasicLayer(
      dim=96, input_resolution=(56, 56), depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (a

In [ ]:
class mySwinTinyP4W7_224(nn.Module):
  def __init__(self, num_classes):
    super(mySwinTinyP4W7_224, self).__init__()
    self.model_ft = create_model('swin_tiny_patch4_window7_224', pretrained=True)
    
    num_ftrs = self.model_ft.head.in_features
    self.model_ft.head = nn.Linear(num_ftrs, num_classes)

  def forward(self, x):
      out = self.model_ft(x)
      return out

In [ ]:
swin_tiny_tf = mySwinTinyP4W7_224(3)

In [ ]:
# train
train_loader, val_loader, test_loader = make_data_loader_with_aug(batch_size=128, size=224)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(swin_tiny_tf.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
MODEL_DIR = '/content/gdrive/MyDrive/ML/MRDC-competition/models'
early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.00001, path=os.path.join(MODEL_DIR, "swin_tiny_224_aug_checkpoint.pt"))

In [ ]:
swin_tiny_tf, val_label, val_pred = train_model(device, swin_tiny_tf, train_loader, val_loader, criterion, optimizer, 30, early_stopping)

Epoch 1/30
----------
train Loss: 0.7604 Acc: 0.6618
val Loss: 0.5450 Acc: 0.7976
Validation loss decreased (inf --> 0.545021).  Saving model ...
Epoch 2/30
----------
train Loss: 0.3130 Acc: 0.8864
val Loss: 0.2668 Acc: 0.9079
Validation loss decreased (0.545021 --> 0.266824).  Saving model ...
Epoch 3/30
----------
train Loss: 0.1191 Acc: 0.9571
val Loss: 0.1960 Acc: 0.9261
Validation loss decreased (0.266824 --> 0.195971).  Saving model ...
Epoch 4/30
----------
train Loss: 0.0886 Acc: 0.9692
val Loss: 0.2916 Acc: 0.9240
EarlyStopping counter: 1 out of 5
Epoch 5/30
----------
train Loss: 0.0795 Acc: 0.9713
val Loss: 0.6265 Acc: 0.8415
EarlyStopping counter: 2 out of 5
Epoch 6/30
----------
train Loss: 0.1062 Acc: 0.9598
val Loss: 0.1567 Acc: 0.9433
Validation loss decreased (0.195971 --> 0.156654).  Saving model ...
Epoch 7/30
----------
train Loss: 0.0903 Acc: 0.9713
val Loss: 0.1468 Acc: 0.9550
Validation loss decreased (0.156654 --> 0.146810).  Saving model ...
Epoch 8/30
-------

In [ ]:
test_pred = test_model(device, swin_tiny_tf, test_loader)

In [ ]:
ss.loc[:, ['blast', 'brown', 'healthy']] = test_pred
ss.to_csv("result_swin_tiny_224_aug.csv", index=False)
ss.head()

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,1.000000,8.112932e-09,1.180154e-09
1,id_01hu05mtch.jpg,0.000475,9.995174e-01,7.347917e-06
2,id_030ln10ewn.jpg,0.996637,2.477106e-03,8.859306e-04
3,id_03z57m8xht.jpg,1.000000,1.022257e-10,1.329884e-11
4,id_04ngep1w4b.jpg,0.970554,2.937046e-02,7.523574e-05


score : 0.36
transformer 기반 모델들은 성능이 다 안좋은듯..

# resnext50d_32x4d (timm ver.)

In [ ]:
m = create_model('resnext50d_32x4d', pretrained=True)
print(m)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnext50d_32x4d-103e99f8.pth" to /root/.cache/torch/hub/checkpoints/resnext50d_32x4d-103e99f8.pth


ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): 

In [ ]:
class myResNext50d(nn.Module):
  def __init__(self, num_classes):
    super(myResNext50d, self).__init__()
    self.model_ft = create_model('resnext50d_32x4d', pretrained=True)
    
    num_ftrs = self.model_ft.fc.in_features
    self.model_ft.fc = nn.Linear(num_ftrs, num_classes)

  def forward(self, x):
      out = self.model_ft(x)
      return out

In [ ]:
resnext50d_tf = myResNext50d(3)

In [ ]:
# train
train_loader, val_loader, test_loader = make_data_loader_with_aug(batch_size=32, size=224)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(resnext50d_tf.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
MODEL_DIR = '/content/gdrive/MyDrive/ML/MRDC-competition/models'
early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.00001, path=os.path.join(MODEL_DIR, "resnext50d_aug_checkpoint.pt"))

In [ ]:
resnext50d_tf, val_label, val_pred = train_model(device, resnext50d_tf, train_loader, val_loader, criterion, optimizer, 30, early_stopping)

Epoch 1/30
----------
train Loss: 0.3949 Acc: 0.8470
val Loss: 0.2090 Acc: 0.9218
Validation loss decreased (inf --> 0.209006).  Saving model ...
Epoch 2/30
----------
train Loss: 0.0974 Acc: 0.9673
val Loss: 0.1643 Acc: 0.9486
Validation loss decreased (0.209006 --> 0.164260).  Saving model ...
Epoch 3/30
----------
train Loss: 0.0859 Acc: 0.9692
val Loss: 0.2007 Acc: 0.9411
EarlyStopping counter: 1 out of 5
Epoch 4/30
----------
train Loss: 0.0854 Acc: 0.9721
val Loss: 0.1512 Acc: 0.9582
Validation loss decreased (0.164260 --> 0.151228).  Saving model ...
Epoch 5/30
----------
train Loss: 0.0993 Acc: 0.9681
val Loss: 0.1337 Acc: 0.9518
Validation loss decreased (0.151228 --> 0.133662).  Saving model ...
Epoch 6/30
----------
train Loss: 0.0683 Acc: 0.9788
val Loss: 0.1016 Acc: 0.9657
Validation loss decreased (0.133662 --> 0.101615).  Saving model ...
Epoch 7/30
----------
train Loss: 0.0465 Acc: 0.9845
val Loss: 0.0951 Acc: 0.9668
Validation loss decreased (0.101615 --> 0.095101).  

In [ ]:
test_pred = test_model(device, resnext50d_tf, test_loader)

In [ ]:
ss.loc[:, ['blast', 'brown', 'healthy']] = test_pred
ss.to_csv("result_resnext50d_aug.csv", index=False)
ss.head()

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.999962,1.767281e-07,3.809216e-05
1,id_01hu05mtch.jpg,0.000003,9.999720e-01,2.532256e-05
2,id_030ln10ewn.jpg,0.175352,8.230674e-01,1.580324e-03
3,id_03z57m8xht.jpg,1.000000,7.598925e-09,1.117548e-08
4,id_04ngep1w4b.jpg,0.997751,2.247754e-03,1.492717e-06


score : 0.15 (SOTA !!)

# resnext101_32x8d

In [ ]:
m = create_model('resnext101_32x8d', pretrained=True)
print(m)

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act3): ReLU(inplace=True)
      (do

In [ ]:
class myResNext101(nn.Module):
  def __init__(self, num_classes):
    super(myResNext101, self).__init__()
    self.model_ft = create_model('resnext101_32x8d', pretrained=True)
    
    num_ftrs = self.model_ft.fc.in_features
    self.model_ft.fc = nn.Linear(num_ftrs, num_classes)

  def forward(self, x):
      out = self.model_ft(x)
      return out

In [ ]:
resnext101_tf = myResNext101(3)

In [ ]:
# train
# train_loader, val_loader, test_loader = make_data_loader_with_aug(batch_size=32, size=224)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(resnext101_tf.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
MODEL_DIR = '/content/gdrive/MyDrive/ML/MRDC-competition/models'
early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.00001, path=os.path.join(MODEL_DIR, "resnext101_aug_checkpoint.pt"))

In [ ]:
resnext101_tf, val_label, val_pred = train_model(device, resnext101_tf, train_loader, val_loader, criterion, optimizer, 30, early_stopping)

Epoch 1/30
----------
train Loss: 0.5143 Acc: 0.7972
val Loss: 2.4462 Acc: 0.7901
Validation loss decreased (inf --> 2.446233).  Saving model ...
Epoch 2/30
----------
train Loss: 0.2295 Acc: 0.9207
val Loss: 0.4034 Acc: 0.8640
Validation loss decreased (2.446233 --> 0.403411).  Saving model ...
Epoch 3/30
----------
train Loss: 0.1376 Acc: 0.9555
val Loss: 0.2570 Acc: 0.9058
Validation loss decreased (0.403411 --> 0.257003).  Saving model ...
Epoch 4/30
----------
train Loss: 0.1193 Acc: 0.9571
val Loss: 0.1838 Acc: 0.9486
Validation loss decreased (0.257003 --> 0.183843).  Saving model ...
Epoch 5/30
----------
train Loss: 0.0714 Acc: 0.9788
val Loss: 0.2047 Acc: 0.9283
EarlyStopping counter: 1 out of 5
Epoch 6/30
----------
train Loss: 0.1040 Acc: 0.9649
val Loss: 0.2416 Acc: 0.9283
EarlyStopping counter: 2 out of 5
Epoch 7/30
----------
train Loss: 0.0555 Acc: 0.9802
val Loss: 0.2501 Acc: 0.9347
EarlyStopping counter: 3 out of 5
Epoch 8/30
----------
train Loss: 0.0701 Acc: 0.9743


In [ ]:
test_pred = test_model(device, resnext101_tf, test_loader)

In [ ]:
ss.loc[:, ['blast', 'brown', 'healthy']] = test_pred
ss.to_csv("result_resnext101_aug.csv", index=False)
ss.head()

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.999959,2.420183e-05,1.632305e-05
1,id_01hu05mtch.jpg,0.000431,9.966742e-01,2.895207e-03
2,id_030ln10ewn.jpg,0.825550,1.377005e-01,3.674957e-02
3,id_03z57m8xht.jpg,1.000000,1.441248e-07,1.858684e-08
4,id_04ngep1w4b.jpg,0.994193,5.440183e-03,3.669000e-04


score : 0.29

# ConvNext Tiny (timm ver.)

In [13]:
class myConvNextTiny(nn.Module):
  def __init__(self, num_classes):
    super( myConvNextTiny, self).__init__()
    self.model_ft = create_model('convnext_tiny', pretrained=True)
    
    num_ftrs = self.model_ft.head.fc.in_features
    self.model_ft.head.fc = nn.Linear(num_ftrs, num_classes)

  def forward(self, x):
      out = self.model_ft(x)
      return out

In [14]:
convnext_tiny_tf = myConvNextTiny(3)

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny_1k_224_ema.pth


In [25]:
# train
train_loader, val_loader, test_loader = make_data_loader_with_aug(batch_size=128, size=224)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(convnext_tiny_tf.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
MODEL_DIR = '/content/gdrive/MyDrive/ML/MRDC-competition/models'
early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.00001, path=os.path.join(MODEL_DIR, "convnext_tiny_aug_checkpoint.pt"))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [26]:
convnext_tiny_tf, val_label, val_pred = train_model(device, convnext_tiny_tf, train_loader, val_loader, criterion, optimizer, 30, early_stopping)

Epoch 1/30
----------
train Loss: 0.5846 Acc: 0.7720
val Loss: 0.3081 Acc: 0.8983
Validation loss decreased (inf --> 0.308103).  Saving model ...
Epoch 2/30
----------
train Loss: 0.1232 Acc: 0.9636
val Loss: 0.1533 Acc: 0.9518
Validation loss decreased (0.308103 --> 0.153276).  Saving model ...
Epoch 3/30
----------
train Loss: 0.0200 Acc: 0.9954
val Loss: 0.1046 Acc: 0.9604
Validation loss decreased (0.153276 --> 0.104606).  Saving model ...
Epoch 4/30
----------
train Loss: 0.0045 Acc: 0.9997
val Loss: 0.1673 Acc: 0.9529
EarlyStopping counter: 1 out of 5
Epoch 5/30
----------
train Loss: 0.0014 Acc: 1.0000
val Loss: 0.1142 Acc: 0.9657
EarlyStopping counter: 2 out of 5
Epoch 6/30
----------
train Loss: 0.0004 Acc: 1.0000
val Loss: 0.1281 Acc: 0.9690
EarlyStopping counter: 3 out of 5
Epoch 7/30
----------
train Loss: 0.0003 Acc: 1.0000
val Loss: 0.1298 Acc: 0.9657
EarlyStopping counter: 4 out of 5
Epoch 8/30
----------
train Loss: 0.0002 Acc: 1.0000
val Loss: 0.1357 Acc: 0.9657
EarlyS

In [27]:
test_pred = test_model(device, convnext_tiny_tf, test_loader)

In [28]:
ss.loc[:, ['blast', 'brown', 'healthy']] = test_pred
ss.to_csv("result_convnext_tiny_timm_ver_aug.csv", index=False)
ss.head()

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.999307,0.000347,0.000346
1,id_01hu05mtch.jpg,0.045972,0.953730,0.000298
2,id_030ln10ewn.jpg,0.941781,0.043251,0.014968
3,id_03z57m8xht.jpg,0.999697,0.000248,0.000055
4,id_04ngep1w4b.jpg,0.998063,0.001426,0.000511


score : 0.22

# resnext50_32x4d (Timm ver.)

In [33]:
class myResNeXt50(nn.Module):
  def __init__(self, num_classes):
    super(myResNeXt50, self).__init__()
    self.model_ft = create_model('resnext50_32x4d', pretrained=True)
    
    num_ftrs = self.model_ft.fc.in_features
    self.model_ft.fc = nn.Linear(num_ftrs, num_classes)

  def forward(self, x):
      out = self.model_ft(x)
      return out

In [34]:
resnext50_tf = myResNeXt50(3)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnext50_32x4d_a1h-0146ab0a.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d_a1h-0146ab0a.pth


In [35]:
# train
train_loader, val_loader, test_loader = make_data_loader_with_aug(batch_size=128, size=224)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(resnext50_tf.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
MODEL_DIR = '/content/gdrive/MyDrive/ML/MRDC-competition/models'
early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.00001, path=os.path.join(MODEL_DIR, "resnext50_aug_checkpoint.pt"))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [36]:
resnext50_tf, val_label, val_pred = train_model(device, resnext50_tf, train_loader, val_loader, criterion, optimizer, 30, early_stopping)

Epoch 1/30
----------
train Loss: 0.8997 Acc: 0.6527
val Loss: 0.5708 Acc: 0.7976
Validation loss decreased (inf --> 0.570780).  Saving model ...
Epoch 2/30
----------
train Loss: 0.3345 Acc: 0.8909
val Loss: 0.3099 Acc: 0.8801
Validation loss decreased (0.570780 --> 0.309892).  Saving model ...
Epoch 3/30
----------
train Loss: 0.0942 Acc: 0.9751
val Loss: 0.1488 Acc: 0.9486
Validation loss decreased (0.309892 --> 0.148824).  Saving model ...
Epoch 4/30
----------
train Loss: 0.0295 Acc: 0.9936
val Loss: 0.1312 Acc: 0.9486
Validation loss decreased (0.148824 --> 0.131169).  Saving model ...
Epoch 5/30
----------
train Loss: 0.0327 Acc: 0.9898
val Loss: 0.1618 Acc: 0.9443
EarlyStopping counter: 1 out of 5
Epoch 6/30
----------
train Loss: 0.0232 Acc: 0.9933
val Loss: 0.2180 Acc: 0.9390
EarlyStopping counter: 2 out of 5
Epoch 7/30
----------
train Loss: 0.0388 Acc: 0.9893
val Loss: 0.1710 Acc: 0.9368
EarlyStopping counter: 3 out of 5
Epoch 8/30
----------
train Loss: 0.0340 Acc: 0.9898


In [37]:
test_pred = test_model(device, resnext50_tf, test_loader)

In [38]:
ss.loc[:, ['blast', 'brown', 'healthy']] = test_pred
ss.to_csv("result_resnext50_timm_ver_aug.csv", index=False)
ss.head()

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.999966,0.000020,0.000013
1,id_01hu05mtch.jpg,0.000458,0.999233,0.000309
2,id_030ln10ewn.jpg,0.618977,0.016132,0.364891
3,id_03z57m8xht.jpg,0.999896,0.000080,0.000024
4,id_04ngep1w4b.jpg,0.996957,0.001783,0.001260


score : 0.4 (엥?)